In [ ]:
# [예제 5.1] folium 모듈을 임포트해 서울 시청을 중심으로 맵 만들기 (

import folium

seoul_map = folium.Map(location=[37.5614378,126.9751701])

In [ ]:
# [예제 5.2] 확대해서 맵 생성하기

seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=14)

In [ ]:
# [예제 5.3] 지도 위에 특정 위치 표시하기

folium.Marker([37.5614378,126.9751701], popup='서울시청').add_to(seoul_map)

In [ ]:
# [예제 5.4] 학교 정보 가져와서 데이터프레임으로 정리하기

import pandas as pd

#school_path = r'C:\Users\JK\Desktop\부동산 데이터\2018년_공시대상학교정보(전체).xlsx'
school_path = r' 학교정보 데이터 디렉터리  학교정보 데이터 엑셀파일명.xlsx'
school_raw = pd.read_excel(school_path)
school_data = school_raw[['정보공시 \n 학교코드', '학교명', '위도', '경도' ]]
school_data.columns = ['학교코드', '학교명','위도', '경도']
school_df = school_data.drop_duplicates()

In [ ]:
school_data

In [ ]:
# [예제 5.5] graduate_preprocrssing 함수 가지고 와 데이터프레임 만들기

def graduate_preprocrssing(path):
    raw_graduate = pd.read_excel(path, sheet_name='2018_졸업생의 진로 현황(중)')
    select_col = raw_graduate[['지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
    select_col.columns = ['지역', '학교명', '학교코드', '졸업자', '과고', '외고']
    graduate_data = select_col.drop(0)
    graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
    graduate_data['외고'] =  pd.to_numeric(graduate_data['외고']) 
    graduate_data['졸업자'] =  pd.to_numeric(graduate_data['졸업자']) 
    graduate_data['총합'] = graduate_data['과고'] + graduate_data['외고']
    
    def get_sido(x):
        temp = x.split(' ')[0]
        if len(temp) != 4:
            return temp[:2]
        else:
            return temp[0] + temp[2]
    
    graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_sido)
    graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])
    
    graduate_data.at[588,'시도'] = '부산'
    graduate_data.at[588,'구군'] = '기장군'
    graduate_data.at[3011,'시도'] = '경북'
    graduate_data.at[3011,'구군'] = '예천군'
    
    return graduate_data

graduate_path = r' 졸업생 진로 데이터 엑셀 파일 디렉터리 \ 졸업생 진로 데이터 엑셀 파일명.xlsx'
gradu_df = graduate_preprocrssing(graduate_path)

In [ ]:
# [예제 5.6] 학교코드를 기준으로 데이터프레임 병합

total_school_df = pd.merge(gradu_df, school_df, how='inner', right_on='학교코드', left_on='학교코드')

In [ ]:
# [예제 5.7] 서울 지역에서 특목고 보낸 중학교 모두 표시하기

seoul_school = total_school_df[total_school_df['시도'] == '서울']
good_school = seoul_school[seoul_school['총합'] > 0]
seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=11)

for n in good_school.index:
    folium.Marker([ good_school['위도'][n], good_school['경도'][n] ], popup=good_school['학교명_x'][n]).add_to(seoul_map)

In [ ]:
# [예제 5.8] 서울의 특목고 진학률이 높은 학교와 낮은 학교 비교

seoul_school['비율'] = seoul_school['총합'] /seoul_school['졸업자'] * 100
good_school = seoul_school[seoul_school['비율'] >= 3]
bad_school = seoul_school[seoul_school['비율'] < 3]
seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=11)

for n in good_school.index:
    folium.CircleMarker([ good_school['위도'][n], good_school['경도'][n] ], color='crimson',fill_color='crimson', radius=7).add_to(seoul_map)

for n in bad_school.index:
    folium.CircleMarker([ bad_school['위도'][n], bad_school['경도'][n] ], color='#3186cc',fill_color='#3186cc',  radius=7).add_to(seoul_map)

In [ ]:
# [예제 5.9] 대전의 특목고 진학률이 높은 학교와 낮은 학교 비교

sido_school = total_school_df[total_school_df['시도'] == '대전']
sido_school['비율'] = sido_school['총합'] /sido_school['졸업자'] * 100
good_school = sido_school[sido_school['비율'] >= 3]
bad_school = sido_school[sido_school['비율'] < 3]
sido_map = folium.Map(location=[36.350461,127.38263], zoom_start=11)

for n in good_school.index:
    folium.CircleMarker([ good_school['위도'][n], good_school['경도'][n] ], color='crimson',fill_color='crimson', radius=7).add_to(sido_map )

for n in bad_school.index:
    folium.CircleMarker([ bad_school['위도'][n], bad_school['경도'][n] ], color='#3186cc',fill_color='#3186cc',  radius=7).add_to(sido_map )

In [ ]:
# [예제 5.10] 법정동 코드 정리

# local_code = pd.read_excel(r'C:\Users\JK\Desktop\부동산 데이터\KIKcd_B.20190101.xlsx')
local_code = pd.read_excel(r' 법정동 코드 데이터 디렉터리 \ 법정동 코드 엑셀 파일명.xlsx')
local_code['법정동코드'] = local_code['법정동코드'].apply(lambda x: str(x)[:5])
filtered_code = local_code[['법정동코드', '시도명', '시군구명']].drop_duplicates()
filtered_code.dropna(inplace=True)
filtered_code.loc[20477] = ('36110', '세종', '세종')

In [ ]:
# [예제 5.11] 법정동 코드 시도명 바꾸기

def get_sido(x):
    temp = x.split(' ')[0]
    if len(temp) != 4:
        return temp[:2]
    else:
        return temp[0] + temp[2]
    
filtered_code['시도명'] = filtered_code['시도명'].dropna().apply(get_sido)
filtered_code['시군구명'] = filtered_code['시군구명'].dropna().apply(lambda x : x.split(' ')[-1])

In [ ]:
# [예제 5.12] KB부동산 데이터에서 매매가격 지수 데이터프레임으로 가져오기

import xlwings as xw

def KBpriceindex_preprocessing(path, data_type):
    # path : KB 데이터 엑셀 파일의 디렉토리 (문자열)
    # data_type : ‘매매종합’, ‘매매APT’, ‘매매연립’, ‘매매단독’, ‘전세종합’, ‘전세APT’, ‘전세연립’, ‘전세단독’ 중 하나
    
    wb = xw.Book(path)                
    sheet = wb.sheets[data_type]   
    row_num = sheet.range(1,1).end('down').end('down').end('down').row  
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index=False, header=True).value 
    
    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])

    for num, gu_data in enumerate(small_col):
        if gu_data == None:
            small_col[num] = big_col[num]

        check = num
        while True:
            if big_col[check] in bigname_list:
                big_col[num] = big_col[check]
                break
            else:
                check = check - 1
                
    big_col[129] = '경기' 
    big_col[130] = '경기'
    small_col[185] = '서귀포'
    
    raw_data.columns = [big_col, small_col]
    new_col_data = raw_data.drop([0,1])
    
    index_list = list(new_col_data['구분']['구분'])

    new_index = []

    for num, raw_index in enumerate(index_list):
        temp = str(raw_index).split('.')
        if int(temp[0]) > 12 :
            if len(temp[0]) == 2:
                new_index.append('19' + temp[0] + '.' + temp[1])
            else:
                new_index.append(temp[0] + '.' + temp[1])
        else:
            new_index.append(new_index[num-1].split('.')[0] + '.' + temp[0])

    new_col_data.set_index(pd.to_datetime(new_index), inplace=True)
    cleaned_data  = new_col_data.drop(('구분', '구분'), axis=1)
    return cleaned_data


path =  r' 여러분이 내려 받은 KB 엑셀파일의 디렉터리를 넣으세요 \ KB엑셀 파일명.xls'
price_index = KBpriceindex_preprocessing(path, '매매apt')

In [ ]:
# [예제 5.13] 매매가격 지수의 증감률 구하여 데이터프레임으로 만들기

diff_pct = ((price_index.loc['2018-6-1']/price_index.loc['2017-6-1']) - 1) * 100
diff_df = pd.DataFrame({'증감률':diff_pct})

In [ ]:
diff_df

In [ ]:
# [예제 5.14] 증감률 데이터프레임에 법정동 코드 추가

filtered_code.index = [filtered_code['시도명'], filtered_code['시군구명']]

code = []
for local in diff_df.index:
    if local[0] in filtered_code.index:
        temp_df = filtered_code.loc[local[0]]
        if local[1] in temp_df.index:
            code.append(temp_df.loc[local[1]]['법정동코드'])
        elif local[1] + '시' in temp_df.index:
            code.append(temp_df.loc[local[1] + '시']['법정동코드'])
        elif local[1] == '세종':
            code.append('36110')
        else:
            code.append('')
    else:
        code.append('')
        
diff_df['법정동코드'] = code

In [ ]:
# [예제 5.15] folium으로 choropleth 매매가격 지수 증감률 지도 만들기 

import json

# rfile = open(r'C:\Users\JK\Desktop\부동산 데이터\TL_SCCO_SIG_WGS84.json', 'r', encoding='utf-8').read()
rfile = open(r' json 파일 디렉터리 \TL_SCCO_SIG_WGS84.json', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)
    
korea_map = folium.Map(location=[36, 127], zoom_start=7)
    
korea_map.choropleth(
 geo_data=jsonData,
 data=diff_df,
 columns=['법정동코드', '증감률'],
 key_on='feature.properties.SIG_CD',
 fill_color='RdYlGn',
 fill_opacity=0.7,
 line_opacity=0.5,
 legend_name='증감률(%)'
)

In [ ]:
# 만들어진 지도 저장하기 

korea_map.save(r'저장할 위치 디렉터리\저장할 이름.html')